In [1]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = '../data/cifar-10-batches-py'
print("1. Structure of CIFAR_DIR {0}".format(os.listdir(CIFAR_DIR)))

1. Structure of CIFAR_DIR ['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def loadData(filename):
    '''
    Read data from filename
    '''
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding = 'bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    '''
    CifarData is a wrapper that process CIFAR-10 Data
    '''
    def __init__(self, filenames, need_shuffle):
        '''
        The constructor takes in a list of files and read all data and labels into self._data member
        @param filenames - list of files that contains all data
        @param need_shuffle - a boolean flag indicating whether order of data needs to be shuffled. 
                              `True` of training set and `False` for testing set
        '''
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = loadData(filename)
            all_data.append(data)
            all_labels.append(labels)
        
        self._data = np.vstack(all_data) / 127.5 - 1
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
            # When we process the data by Mini Batch, this is a cursor indicating where
            # we are in the dataset
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        '''
        Get a random permutation of all indices between 0 and _num_examples, and apply 
        this random permutation of indices to `self._data` and `self._labels`.
        '''
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        '''
        Generate a batch of data for processing. For training set, if end index is larger 
        than `self._num_examples`, reshuffle the data and start over from index 0. For 
        testing set, if end index is out of bound, raise an exception.
        @param batch_size - Number of samples for the batch of data and labels to return
        '''
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("Do not have enough testing samples for next batch")
                
        if end_indicator > self._num_examples:
            raise Exception("Batch size is larger than _num_examples")
            
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    def reset(self):
        '''
        `_shuffle_data` and reset `_indicator` to 0
        '''
        self._shuffle_data()
        self._indicator = 0
    
    def getDataShape(self):
        return self._data.shape
    
    def getLabelShape(self):
        return self._labels.shape
    
print("2. Basic test for `loadData` and `CifarData` class.")
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range (1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
print("Train data shape = {0}, labels shape = {1}".
      format(train_data.getDataShape(), train_data.getLabelShape()))

test_data = CifarData(test_filenames, False)
print("Test data shape = {0}, labels shape = {1}".
      format(test_data.getDataShape(), test_data.getLabelShape()))

batch_data, batch_labels = train_data.next_batch(10)
print("Batch Data = {0}, Batch Labels = {1}".format(batch_data, batch_labels))

2. Basic test for `loadData` and `CifarData` class.
Train data shape = (50000, 3072), labels shape = (50000,)
Test data shape = (10000, 3072), labels shape = (10000,)
Batch Data = [[-0.06666667 -0.05098039 -0.02745098 ... -0.41960784 -0.41960784
  -0.42745098]
 [ 1.          0.99215686  0.99215686 ...  0.63921569  0.63137255
   0.60784314]
 [-0.77254902 -0.75686275 -0.71764706 ... -0.25490196 -0.30196078
  -0.3254902 ]
 ...
 [-0.65490196 -0.6627451  -0.74901961 ...  0.08235294  0.09803922
   0.1372549 ]
 [ 0.12941176  0.10588235  0.11372549 ... -0.05098039 -0.05098039
  -0.05098039]
 [-0.92156863 -0.94509804 -0.92941176 ...  0.0745098  -0.12156863
  -0.05882353]], Batch Labels = [0 8 7 8 0 7 1 7 6 8]


In [3]:
# Define x and y
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

hidden1 = tf.layers.dense(x, 100, activation = tf.nn.relu)
hidden2 = tf.layers.dense(hidden1, 100, activation = tf.nn.relu)
hidden3 = tf.layers.dense(hidden2, 50, activation = tf.nn.relu)
z = tf.layers.dense(hidden3, 10)
y_prob = tf.nn.softmax(z)
y_predict = tf.argmax(z, 1)
y_actual_oneHot = tf.one_hot(y, 10, dtype = tf.float32)

# Calculate loss and accuracy
loss = tf.reduce_mean(tf.square(y_actual_oneHot - y_prob))
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_predict, y), tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
print("3. Apply TensorFlow model to CIFAR-10 Data.")
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100
train_data.reset()

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op],
                                        feed_dict = {x: batch_data, y: batch_labels})
        # Every 500 steps print loss and accuracy
        if (i + 1) % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, accuracy: %4.5f' \
                 % (i + 1, loss_val, acc_val))
        
        # Every 5,000 steps run model against test model for accuracy
        if (i + 1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_accuracies = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], 
                                        feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_accuracies.append(test_acc_val)
            test_accuracy = np.mean(all_test_accuracies)
            print('[Test ] step: %d, accuracy: %4.5f' % (i + 1, test_accuracy))

3. Apply TensorFlow model to CIFAR-10 Data.
[Train] step: 500, loss: 0.07226, accuracy: 0.35000
[Train] step: 1000, loss: 0.07369, accuracy: 0.40000
[Train] step: 1500, loss: 0.08002, accuracy: 0.40000
[Train] step: 2000, loss: 0.07815, accuracy: 0.45000
[Train] step: 2500, loss: 0.06952, accuracy: 0.50000
[Train] step: 3000, loss: 0.06011, accuracy: 0.55000
[Train] step: 3500, loss: 0.07174, accuracy: 0.40000
[Train] step: 4000, loss: 0.06392, accuracy: 0.55000
[Train] step: 4500, loss: 0.07488, accuracy: 0.45000
[Train] step: 5000, loss: 0.06788, accuracy: 0.45000
[Test ] step: 5000, accuracy: 0.46900
[Train] step: 5500, loss: 0.06471, accuracy: 0.40000
[Train] step: 6000, loss: 0.07779, accuracy: 0.40000
[Train] step: 6500, loss: 0.06159, accuracy: 0.45000
[Train] step: 7000, loss: 0.06138, accuracy: 0.50000
[Train] step: 7500, loss: 0.05849, accuracy: 0.55000
[Train] step: 8000, loss: 0.09441, accuracy: 0.30000
[Train] step: 8500, loss: 0.05739, accuracy: 0.60000
[Train] step: 9000